In [35]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [42]:
base_model=MobileNet(weights='imagenet',include_top=False,input_shape=(128,128,3)) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(1,activation='sigmoid')(x) #final layer with softmax activation

In [43]:
model=Model(inputs=base_model.input,outputs=preds)

In [44]:
print(len(model.layers))

92


In [45]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [46]:
image_data_generator=ImageDataGenerator(
                        rescale=1./255,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True)

train_generator=image_data_generator.flow_from_directory('data/train/', # this is where you specify the path to the main data folder
                                                 target_size=(128,128),
                                                 batch_size=32,
                                                 class_mode='binary',
                                                 shuffle=True)

Found 418 images belonging to 2 classes.


In [51]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=3)

Epoch 1/3
13/13 [==============================] - 37s 3s/step - loss: 0.1028 - acc: 0.9903
Epoch 2/3
13/13 [==============================] - 24s 2s/step - loss: 0.4571 - acc: 0.9423
Epoch 3/3
13/13 [==============================] - 24s 2s/step - loss: 0.1014 - acc: 0.9637


In [52]:
test_datagenerator = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagenerator.flow_from_directory(
        'data/validation/',
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary')

Found 399 images belonging to 2 classes.


In [53]:
model.metrics_names
result=model.evaluate_generator(generator=validation_generator,steps=50)

In [54]:
print("Model loss is %f accuracy is %f "%(result[0],result[1]))

Model loss is 2.233724 accuracy is 0.679148 
